In [27]:
import gymnasium as gym
import matplotlib
import matplotlib.pyplot as plt
from IPython.display import clear_output

import torch
import torch.nn as nn
import torch.nn.functional as F

from importlib import reload
from copy import deepcopy
from typing import List
from collections import deque, namedtuple

import random
import math

import utils.plots_cliffwalking as plots

# set up matplotlib
is_ipython = 'inline' in matplotlib.get_backend()
if is_ipython:
    from IPython import display

plt.ion()


## Criando o modelo

No Deep Q-Learning, os Q-valores de cada ação associados a um estado são calculados através de uma rede neural. Ou seja, a rede neural recebe como entrada um vetor de estados e deve retornar o vetor de Q-valores onde cada elemento representa o Q-valor de uma ação. Um Q-valor pode ser interpretado como "a recompensa acumulada total esperada por executar a ação A no estado S e depois seguir a mesma política até o final do episódio".

Por se tratar de um problema relativamente simples, o modelo para solucionar o CliffWalking pode ser uma rede MLP. Além disso, note que a predição dos Q-valores é uma tarefa de regressão, portanto não é utilizada uma função de ativação softmax no final da rede. 

In [28]:
class Qnet(torch.nn.Module):
    def __init__(self, layer_sizes: List[int] = [32]):
        super().__init__()

        # construindo a rede neural
        layers = []
        input_size = 4 # entrada: posicao (x, y)
        for n_neurons in layer_sizes:
            layers.append(torch.nn.Linear(input_size, n_neurons))
            layers.append(torch.nn.ReLU())
            input_size = n_neurons
        layers.append(torch.nn.Linear(input_size, 2))
        self.nn = torch.nn.Sequential(*layers)

    def _encode(self, x):
        return torch.tensor(x, dtype=torch.float32)

    def forward(self, x):
        x = self._encode(x)
        x = self.nn(x)
        return x

## Replay buffer

Replay buffers são utilizados (...)


In [29]:
Transition = namedtuple('Transition', ('state', 'action', 'reward', 'next_state', 'terminated'))

class ReplayBuffer(object):
    def __init__(self, capacity):
        self.memory = deque([], maxlen=capacity)

    def push(self, *args):
        """Save a transition"""
        self.memory.append(Transition(*args))

    def sample(self, batch_size):
        return random.sample(self.memory, batch_size)

    def __len__(self):
        return len(self.memory)


## Amostrando ações com a política $\epsilon$-greedy

No final do treinamento, espera-se que a melhor ação para cada estado seja aquela cujo Q-Valor é o maior. No entanto, para que o Q-Learning convirja adequadamente, é necessário que no início do treinamento o agente "explore" bem o ambiente. Isto é, que o agente visite um grande número de estados mesmo que não sejam necessariamente ótimos. Uma técnica amplamente utilizada para essa finalidade é a política $\epsilon$-greedy. Ela consiste em forçar o agente a escolher ações aleatoriamente com uma frequência que diminui conforme o treinamento avança.

In [30]:
@torch.no_grad()
def get_action(model, state, epsilon=1, n_actions=2):
    if torch.rand(1) < epsilon:
        return torch.randint(n_actions, (1,)).item()
    qvals = model(state)
    return torch.argmax(qvals).item()

## Treinamento da rede neural 

A cada passo do treinamento, o agente executará uma ação e utilizará a informação retornada pelo ambiente para calcular uma loss e atualizar seus pesos de forma a minimizá-la. A loss que será utilizada é o erro quadrático médio entre o Q-valor escolhido e o maior Q-valor do próximo estado calculado utilizando a rede com os pesos anteriores à ultima atualização:

$$L_i(\theta_i)=\mathbb{E}[(y_i - Q(s,a;\theta_i))^2]$$
$$y_i=\mathbb{E}[R(s')+\gamma\max_A Q(s',A;\theta_{i-1})]$$

Note que, portanto, serão necessárias duas redes neurais com a mesma arquitetura, mas uma terá os pesos deefasados em uma iteração com relação à outra.

In [31]:
def update_q_net(model: Qnet, 
                 model_target: Qnet,
                 optimizer: torch.optim.Optimizer, 
                 batch_of_transitions: List[Transition],
                 gamma):
    
    # convertendo uma lista de Transitions para uma Transition de listas
    # mais informacoes: https://stackoverflow.com/questions/19339/transpose-unzip-function-inverse-of-zip/19343#19343
    batch = Transition(*zip(*batch_of_transitions))
    
    # transformando o batch de estados em um tensor
    states = torch.tensor(batch.state)
    actions = torch.tensor(batch.action)
    rewards = torch.tensor(batch.reward)
    next_states = torch.tensor(batch.next_state)
    terminated = torch.tensor(batch.terminated)
    
    predictions = model(states).gather(1, actions.unsqueeze(1)).squeeze() # seleciona o qval da acao tomada
    with torch.no_grad():
        targets = rewards + gamma * model_target(next_states).max(-1).values * (1 - terminated.int())
    
    loss = F.mse_loss(predictions, targets)

    # atualizando os pesos da rede
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    return model

## Loop de treinamento

No loop de treinamento, juntaremos todas as funções desenvolvidas até o momento. A ideia principal é definir um número máximo de episódios (estágio inicial até o estágio final) para que o agente colete experiências do ambiente e otimize sua tabela de QValores.

In [32]:
def evaluate(metrics, show_result=False):
    # duration of each episode

    # durations_t = torch.tensor(episode_durations, dtype=torch.float)
    durations = torch.tensor(metrics['episode_durations'], dtype=torch.float)
    plt.figure(1)
    plt.clf
    plt.title('Duration of each episode')
    plt.xlabel('Episode')
    plt.ylabel('Duration')
    plt.plot(durations)

    # Take 100 episode averages and plot them too
    if len(durations) >= 100:
        means = durations.unfold(0, 100, 1).mean(-1).view(-1)
        means = torch.cat((torch.zeros(99), means))
        plt.plot(means.numpy())
    plt.pause(0.001)  # pause a bit so that plots are updated

    if not show_result:
        display.display(plt.gcf())
        display.clear_output(wait=True)
    else:
        display.display(plt.gcf())

In [33]:
debug_model = None

def train(
        env: gym.Env, 
        model: Qnet,
        total_timesteps=500000, # numero maximo de episodios
        learning_rate=1e-4, # taxa de aprendizado
        replay_buffer_size=10_000, # tamanho do replay buffer
        gamma=0.99, # fator de desconto
        tau = 1.0, # fator de mistura para atualizacao da rede defasada
        target_update_freq = 1, # frequencia de atualizacao da rede defasada
        batch_size=128, # tamanho do batch
        epsilon_0 = 1, # probabilidade inicial de escolher uma ação aleatória
        epsilon_f=0.05, # probabilidade final de escolher uma ação aleatória (após o decaimento)
        epsilon_decay = 1000, # step in which epsilon will be approximately epsilon_f + 0.36 * (epsilon_0 - epsilon_f)
        # learn_starts_in = 10000, # numero de timesteps antes de comecar a treinar
        learning_freq = 1, # frequencia de treinamento
        verbose=False):

    model_target = deepcopy(model)

    def eps_scheduler(step):
        return epsilon_f + (epsilon_0 - epsilon_f) * math.exp(-1. * step / epsilon_decay)

    replay_buffer = ReplayBuffer(replay_buffer_size)
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)    
    state, _ = env.reset()
    metrics = {
        'episode_durations': [],
    }
    episode_steps = 0

    for global_step in range(total_timesteps):
        epsilon = eps_scheduler(global_step)

        # observe
        action = get_action(model, state, epsilon)
        next_state, reward, terminated, truncated, _ = env.step(action)
        replay_buffer.push(state, action, reward, next_state, terminated)
        state = next_state
        episode_steps += 1
        done = terminated or truncated

        # update
        if global_step > batch_size and global_step % learning_freq == 0:
            batch = replay_buffer.sample(batch_size)
            model = update_q_net(model, model_target, optimizer, batch, gamma)

        # update target network
        if global_step % target_update_freq == 0:
            model_target.load_state_dict(model.state_dict())

        # evaluating
        if done:
            metrics['episode_durations'].append(episode_steps)
            episode_steps = 0
            print(f'current step: {global_step}, epsilon: {epsilon}, episode: {len(metrics["episode_durations"])}')
            state, _ = env.reset()
            evaluate(metrics, show_result=False)

    return model

## Treinando

Está tudo configurado, portanto agora podemos rodar o algoritmo!

In [35]:
torch.manual_seed(0)
random.seed(0)
cartpole = gym.make('CartPole-v1')
q_net = Qnet(layer_sizes=[128, 128])
debug_model = deepcopy(q_net)
trained_q_net = train(cartpole, q_net, total_timesteps=15000, verbose=True)

KeyboardInterrupt: 

<Figure size 640x480 with 0 Axes>

## Testando o agente

A função abaixo rodará um episódio com o agente já treinado.

In [ ]:
def test(env: gym.Env, 
          q_net,
          n_episodes=1,
          verbose=False
          ):
    
    total_rewards = []
    for episode in range(n_episodes):
        state, _ = env.reset()
        total_reward = 0
        done = False
        
        while not done:
            action = get_action(q_net, state, 0)
            state, reward, terminated, truncated, _ = env.step(action)
            total_reward += reward

            done = terminated or truncated

        if verbose:
            print(f"Episode {episode} - Total reward: {total_reward}")
            
        total_rewards.append(total_reward)

    env.close()
    return torch.mean(total_reward)

test(gym.make('CartPole-v1', render_mode="human"), trained_q_net)

TypeError: mean(): argument 'input' (position 1) must be Tensor, not float